In [1]:
import pandas as pd

df = pd.read_csv('/content/IOT-temp 2.csv')

df=df.drop(columns = ['id', 'room_id/id', 'noted_date'])
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
#df = imputer.fit_transform(df)

from sklearn.preprocessing import LabelEncoder
le1= LabelEncoder()


df['out/in']= le1.fit_transform(df['out/in'])

'''
df=df.dropna()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
le2 = LabelEncoder()
'''


#df['icon']= le.fit_transform(df['icon'])

y = df['out/in'].values
df=df.drop(columns = ['out/in'])
X = df.values


In [ ]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
df.shape


(594, 19)

In [ ]:
from collections import Counter
Counter(y)

Counter({0: 20345, 1: 77261})

In [2]:

y=y.reshape(-1, 1)


import numpy as np
X=np.concatenate((X[1:], y[0:-1]), axis=1)

In [1]:
pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=1fe27358151aa3f10e6a50c9999429b53a0b6c6855aa9b3c941a07da47be124f
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X= sc.fit_transform(X)

In [2]:
pip install pyrcn

     |████████████████████████████████| 81 kB 4.3 MB/s 


In [4]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(X, y[1:], test_size=0.2, shuffle=False, random_state=42)


In [ ]:
x_train1.shape

(78084, 2)

In [5]:
from pyrcn.base.blocks import InputToNode
from sklearn . datasets import make_blobs
# Generate a toy dataset

In [6]:
input_to_node = InputToNode (8, input_activation='relu',input_scaling =1.0 )


x_train= input_to_node.fit_transform (x_train1)
x_test= input_to_node.transform (x_test1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:77: FutureWarning: Pass hidden_layer_size=8 as keyword args. From version 1.1 (renaming of 0.26) passing these as positional arguments will result in an error
  FutureWarning,


In [7]:
from pyrcn.base.blocks import NodeToNode
node_to_node = NodeToNode (8, reservoir_activation='relu', spectral_radius =1.0 , leakage =0.4 ,bidirectional = False )
x_train=node_to_node . fit_transform(x_train)
x_test= node_to_node.transform (x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:77: FutureWarning: Pass hidden_layer_size=8 as keyword args. From version 1.1 (renaming of 0.26) passing these as positional arguments will result in an error
  FutureWarning,


In [8]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

x_train1 = x_train1.reshape((x_train1.shape[0], x_train1.shape[1], 1))
x_test1 = x_test1.reshape((x_test1.shape[0], x_test1.shape[1], 1))

In [9]:
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [11]:
from keras_self_attention import SeqSelfAttention
def build_model(
    input_shapey,
    input_shapez,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputsy = keras.Input(shape=input_shapey)
    inputsz = keras.Input(shape=input_shapez)
    y = inputsy
    z= inputsz

    #z=layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(z)

    #y=layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(y)

    z=SeqSelfAttention()(z)

    #y=SeqSelfAttention()(y)



    x=layers.Concatenate(axis=1)([y, z])
    #x=layers.Add()([inputsy, inputsz])

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(2, activation="softmax")(x)
    #outputs = layers.Dense(1)(x)
    return keras.Model([inputsy,inputsz], outputs)

In [12]:
input_shapey = x_train1.shape[1:]
input_shapez = x_train.shape[1:]

model = build_model(
    input_shapey,
    input_shapez,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)
'''
model.compile(
    loss="mean_absolute_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["mean_absolute_error"],
)
'''
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["sparse_categorical_accuracy"],
)

model.summary()





Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8, 1)]       0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 2, 1)]       0           []                               
                                                                                                  
 seq_self_attention (SeqSelfAtt  (None, 8, 1)        129         ['input_2[0][0]']                
 ention)                                                                                          
                                                                                                  
 concatenate (Concatenate)      (None, 10, 1)        0           ['input_1[0][0]',            

In [13]:


callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history=model.fit(
    [x_train1,x_train],
    y_train1,
    validation_data=([x_test1, x_test], y_test1),
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
)

Epoch 1/100
611/611 [==============================] - 17s 19ms/step - loss: 0.2656 - sparse_categorical_accuracy: 0.8783 - val_loss: 0.5145 - val_sparse_categorical_accuracy: 0.7327
Epoch 2/100
611/611 [==============================] - 11s 18ms/step - loss: 0.1857 - sparse_categorical_accuracy: 0.9131 - val_loss: 0.4788 - val_sparse_categorical_accuracy: 0.7412
Epoch 3/100
611/611 [==============================] - 11s 18ms/step - loss: 0.1584 - sparse_categorical_accuracy: 0.9289 - val_loss: 0.4649 - val_sparse_categorical_accuracy: 0.7532
Epoch 4/100
611/611 [==============================] - 11s 18ms/step - loss: 0.1453 - sparse_categorical_accuracy: 0.9326 - val_loss: 0.4610 - val_sparse_categorical_accuracy: 0.7674
Epoch 5/100
611/611 [==============================] - 11s 18ms/step - loss: 0.1409 - sparse_categorical_accuracy: 0.9343 - val_loss: 0.4615 - val_sparse_categorical_accuracy: 0.7708
Epoch 6/100
611/611 [==============================] - 11s 18ms/step - loss: 0.1384 -

In [14]:
#testing prediction

#testing prediction
ypred1=model.predict([x_test1, x_test])

ypred=[]
#testing prediction
for i in ypred1:
    if i[0]>i[1]:
        ypred.append(0)
    else:
        ypred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print(accuracy_score(ypred, y_test1))



print('f1_score', f1_score(ypred, y_test1))



print('jaccard_score', jaccard_score(ypred, y_test1))
print('roc_auc_score', roc_auc_score(ypred, y_test1))
print('precision_score', precision_score(ypred, y_test1))

print('recall_score', recall_score(ypred, y_test1))


0.7752676604682137
f1_score 0.8278460149903857
jaccard_score 0.7062604620020088
roc_auc_score 0.7720873933912126
precision_score 0.8814975764666555
recall_score 0.780350669527262


In [15]:
#training prediction
xpred1=model.predict([x_train1, x_train])
xpred=[]

for i in xpred1:
    if i[0]>i[1]:
        xpred.append(0)
    else:
        xpred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print('accuracy', accuracy_score(xpred, y_train1))



print('f1_score', f1_score(xpred, y_train1))



print('jaccard_score', jaccard_score(xpred, y_train1))
print('roc_auc_score', roc_auc_score(xpred, y_train1))
print('precision_score', precision_score(xpred, y_train1))

print('recall_score', recall_score(xpred, y_train1))


accuracy 0.9451232006557041
f1_score 0.9676859846913767
jaccard_score 0.93739498867704
roc_auc_score 0.9239277003461979
precision_score 0.9826173520177656
recall_score 0.9532016045164166
